# Microsoft Fabric ML Predictive Maintenance
## Predict Component Replacement Using Machine Learning

**Objectives:**
1. Upload historical CSV data to Microsoft Fabric Lakehouse
2. Create features for machine learning
3. Train predictive model using flaml.AutoML
4. Generate predictions and save to Lakehouse
5. Visualize ML predictions in Power BI

**Prerequisites:**
- Microsoft Fabric workspace with Lakehouse named `MaintenanceML`
- CSV file: `quality_30days.csv` uploaded to Lakehouse Files
- This notebook attached to `MaintenanceML` Lakehouse

## Part 1: Upload Data to Fabric Lakehouse

**Manual Steps (before running this notebook):**

1. Go to Microsoft Fabric portal: https://app.fabric.microsoft.com
2. Navigate to your workspace → **+ New** → **Lakehouse**
3. Name: `MaintenanceML` → **Create**
4. Click **Get data** → **Upload files**
5. Select `quality_30days.csv` → Wait for upload
6. Attach this notebook to the Lakehouse: **Add** → select `MaintenanceML`

---

## Part 1.1: Import Required Libraries

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

## Part 1.2: Load CSV to Lakehouse Table

In [ ]:
# Load CSV from Lakehouse Files into a Spark DataFrame
csv_path = "Files/quality_30days.csv"
df_spark = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_path)

# Save to Lakehouse table
df_spark.write.mode("overwrite").saveAsTable("MaintenanceML.machine_data_raw")

print(f"✅ CSV loaded to table: MaintenanceML.machine_data_raw")
print(f"Total rows: {df_spark.count():,}")
df_spark.printSchema()

## Part 2: Load and Prepare Data

In [ ]:
# Load raw data from Lakehouse table
df = spark.read.table("MaintenanceML.machine_data_raw").toPandas()

# Convert timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(['MachineID', 'Timestamp']).reset_index(drop=True)

print(f"Loaded {len(df):,} records")
print(f"Date range: {df['Timestamp'].min()} to {df['Timestamp'].max()}")
print(f"Machines: {df['MachineID'].nunique()}")

## Part 3: Calculate Rotation Features

In [ ]:
# Calculate rotation count per cycle
df['RotationCount'] = df['ActualAngle'] / 360.0

# Cumulative rotations per machine (key wear indicator)
df['CumulativeRotation'] = df.groupby('MachineID')['RotationCount'].cumsum()

# Cumulative bit rotations (use BitRotationCounter if available)
if 'BitRotationCounter' in df.columns:
    df['CumulativeBitRotation'] = df['BitRotationCounter']
else:
    df['CumulativeBitRotation'] = df['CumulativeRotation']

print("✅ Rotation features created")
df[['Timestamp', 'MachineID', 'RotationCount', 'CumulativeBitRotation']].head()

## Part 4: Create Rolling Window Features

In [ ]:
# Set timestamp as index for rolling calculations
df_indexed = df.set_index('Timestamp')

# Create rolling features (1 hour window)
rolling_features = df_indexed.groupby('MachineID').rolling('1H').agg({
    'RotationCount': ['sum', 'mean'],
    'ActualTorque': ['mean', 'std'],
    'CycleTime_ms': ['mean', 'max'],
    'CycleOK': 'mean'  # Pass rate in last hour
}).reset_index()

# Flatten column names
rolling_features.columns = ['MachineID', 'Timestamp', 
                             'Rot_LastHour_Sum', 'Rot_LastHour_Avg',
                             'Torque_LastHour_Avg', 'Torque_LastHour_Std',
                             'CycleTime_LastHour_Avg', 'CycleTime_LastHour_Max',
                             'PassRate_LastHour']

# Merge back to main dataframe
df = df.merge(rolling_features, on=['MachineID', 'Timestamp'], how='left')

# Fill NaN with 0 for first hour
df = df.fillna(0)

print("✅ Rolling window features created")
df[['MachineID', 'Rot_LastHour_Sum', 'Torque_LastHour_Avg', 'PassRate_LastHour']].head()

## Part 5: Create Target Variable (Remaining Rotations)

In [ ]:
# Define component lifetime (from reference data)
BIT_LIFETIME = 100000  # rotations

# Calculate remaining rotations until replacement
df['RemainingRotations'] = BIT_LIFETIME - df['CumulativeBitRotation']

# Calculate days until replacement (assuming current rotation rate)
df['RotationsPerHour'] = df['Rot_LastHour_Sum']
df['HoursUntilReplacement'] = df['RemainingRotations'] / (df['RotationsPerHour'] + 0.1)  # avoid divide by zero
df['DaysUntilReplacement'] = df['HoursUntilReplacement'] / 24

# Clip predictions to reasonable range (0-365 days)
df['DaysUntilReplacement'] = df['DaysUntilReplacement'].clip(0, 365)

print("✅ Target variable created")
print(f"Average days until replacement: {df['DaysUntilReplacement'].mean():.1f}")
df[['MachineID', 'CumulativeBitRotation', 'RemainingRotations', 'DaysUntilReplacement']].tail(10)

## Part 6: Save Feature Table to Lakehouse

In [ ]:
# Select features for ML model
feature_columns = [
    'Timestamp', 'MachineID', 'ProductID', 'ScrewPosition',
    'CumulativeBitRotation', 'RotationCount',
    'Rot_LastHour_Sum', 'Rot_LastHour_Avg',
    'Torque_LastHour_Avg', 'Torque_LastHour_Std',
    'CycleTime_LastHour_Avg', 'CycleTime_LastHour_Max',
    'PassRate_LastHour',
    'ActualTorque', 'ActualAngle', 'CycleTime_ms',
    'DaysUntilReplacement'  # Target variable
]

df_features = df[feature_columns].copy()

# Convert back to Spark DataFrame and save to Lakehouse
spark_df = spark.createDataFrame(df_features)
spark_df.write.mode("overwrite").saveAsTable("MaintenanceML.ml_features")

print("✅ Feature table saved to Lakehouse: ml_features")
print(f"Total rows: {len(df_features):,}")
print(f"Feature columns: {len(feature_columns)}")

## Part 7: Train ML Model with flaml.AutoML

**Using Microsoft Fabric AutoML (flaml library)**

According to Microsoft Fabric documentation, use `flaml.AutoML` for code-first automated machine learning.

In [ ]:
# Train ML Model using flaml.AutoML (Fabric-native AutoML)
import mlflow
from flaml import AutoML

# Set up MLflow experiment
EXPERIMENT_NAME = "Predictive_Maintenance_Experiment"
mlflow.set_experiment(EXPERIMENT_NAME)

# Prepare training data
feature_cols = [
    'CumulativeBitRotation', 'RotationCount',
    'Rot_LastHour_Sum', 'Rot_LastHour_Avg',
    'Torque_LastHour_Avg', 'Torque_LastHour_Std',
    'CycleTime_LastHour_Avg', 'CycleTime_LastHour_Max',
    'PassRate_LastHour',
    'ActualTorque', 'ActualAngle', 'CycleTime_ms'
]

X_train = df_features[feature_cols]
y_train = df_features['DaysUntilReplacement']

# Initialize AutoML
automl = AutoML()

# Configure AutoML settings
settings = {
    "time_budget": 300,  # 5 minutes training time
    "metric": "r2",  # R-squared for regression
    "task": "regression",
    "log_file_name": "automl_maintenance.log",
    "seed": 12345
}

# Start AutoML training with MLflow logging
with mlflow.start_run(run_name="AutoML_Predictive_Maintenance") as run:
    automl.fit(X_train=X_train, y_train=y_train, **settings)
    
    # Get the best model (the actual estimator, not the AutoML wrapper)
    best_model = automl.model
    
    # Log best model metrics
    mlflow.log_metric("best_r2", automl.best_loss)
    mlflow.log_metric("best_estimator_r2", 1 - automl.best_loss)
    mlflow.log_param("best_estimator", str(automl.best_estimator))
    mlflow.log_param("best_config", str(automl.best_config))
    
    # Log the actual trained model (not the AutoML wrapper)
    mlflow.sklearn.log_model(best_model, "model")
    
    print(f"✅ Best estimator: {automl.best_estimator}")
    print(f"✅ Best loss: {automl.best_loss:.4f}")
    print(f"✅ Best R²: {1 - automl.best_loss:.4f}")
    print(f"✅ Model saved to MLflow experiment: {EXPERIMENT_NAME}")
    print(f"✅ Run ID: {run.info.run_id}")

## Part 8: Load Trained Model from MLflow

In [ ]:
# Load the trained model from MLflow
loaded_model = mlflow.sklearn.load_model(f"runs:/{run.info.run_id}/model")

print(f"✅ Model loaded successfully from run: {run.info.run_id}")

## Part 9: Prepare Latest Data for Prediction

In [ ]:
# Load feature table
df_features_full = spark.read.table("MaintenanceML.ml_features").toPandas()

# Get latest state per machine (most recent timestamp)
df_latest = df_features_full.sort_values('Timestamp').groupby('MachineID').tail(1).reset_index(drop=True)

# Select feature columns (exclude target and identifiers)
X_predict = df_latest[feature_cols]

print(f"Predicting for {len(df_latest)} machines")
df_latest[['MachineID', 'Timestamp', 'CumulativeBitRotation']].head()

## Part 10: Generate Predictions

In [ ]:
# Make predictions
predictions = loaded_model.predict(X_predict)

# Add predictions to dataframe
df_latest['ML_PredictedDays'] = predictions

# Calculate confidence/risk level
df_latest['RiskLevel'] = pd.cut(
    df_latest['ML_PredictedDays'],
    bins=[0, 2, 7, 14, 365],
    labels=['🔴 CRITICAL', '🟠 URGENT', '🟡 WARNING', '🟢 GOOD']
)

# Add prediction timestamp
df_latest['PredictionTimestamp'] = datetime.utcnow()

print("✅ Predictions generated")
df_latest[['MachineID', 'ML_PredictedDays', 'RiskLevel', 'CumulativeBitRotation']].head(10)

## Part 10.1: Load New CSV Data for Prediction (Optional)

**Use this section to predict on new CSV data from IoT simulator**

## Part 11: Save Predictions to Lakehouse

In [ ]:
# Select columns for prediction table
prediction_cols = [
    'PredictionTimestamp', 'Timestamp', 'MachineID',
    'CumulativeBitRotation', 'RotationCount',
    'Rot_LastHour_Sum', 'PassRate_LastHour',
    'DaysUntilReplacement',  # Rule-based prediction
    'ML_PredictedDays',       # ML prediction
    'RiskLevel'
]

df_predictions = df_latest[prediction_cols].copy()

# Convert to Spark and save
spark_predictions = spark.createDataFrame(df_predictions)
spark_predictions.write.mode("overwrite").saveAsTable("MaintenanceML.ml_predictions")

print("✅ Predictions saved to: MaintenanceML.ml_predictions")
print(f"Total predictions: {len(df_predictions)}")
print("\n📊 Risk Distribution:")
print(df_predictions['RiskLevel'].value_counts())

## Part 12: Connect Power BI to Lakehouse (Instructions)

**Manual Steps in Power BI Desktop:**

1. Open **Power BI Desktop**
2. Click **Home** → **Get Data** → **More**
3. Search for: **Microsoft Fabric**
4. Select **Lakehouse** → **Connect**
5. Sign in with your Fabric credentials
6. Navigate to workspace → Select `MaintenanceML` lakehouse
7. Select tables:
   - ✅ `ml_predictions`
   - ✅ `machine_data_raw` (optional)
8. Click **Load**

---

## Part 13: DAX Measures for Power BI (Reference)

**Create these measures in Power BI:**

```dax
// Average ML Prediction
AvgML_PredictedDays = AVERAGE(ml_predictions[ML_PredictedDays])

// Average Rule-Based Prediction
AvgRule_PredictedDays = AVERAGE(ml_predictions[DaysUntilReplacement])

// Prediction Difference
PredictionDifference = [AvgML_PredictedDays] - [AvgRule_PredictedDays]

// Critical Machines Count
CriticalMachines = 
CALCULATE(
    DISTINCTCOUNT(ml_predictions[MachineID]),
    ml_predictions[RiskLevel] = "🔴 CRITICAL"
)
```

**Dashboard Visuals:**
1. KPI Cards: Avg ML Days, Avg Rule Days, Prediction Difference, Critical Machines
2. Matrix: Machine Risk Status (rows: MachineID, columns: RiskLevel)
3. Bar Chart: ML vs Rule-Based by Machine
4. Scatter Plot: Rotation vs Predicted Days

---

## Part 14: Schedule Automated Predictions (Optional)

**Create Data Pipeline in Microsoft Fabric:**

1. In Fabric workspace: **+ New** → **Data pipeline**
2. Name: `Daily_ML_Predictions`
3. Add activities:
   - **Notebook** → Select this notebook
   - Configure: Run all cells
4. Set schedule:
   - **Schedule** tab
   - Frequency: Daily
   - Time: 1:00 AM UTC
5. Click **Publish**

**Alternative: Use this code to run specific sections**

In [ ]:
# Automated daily prediction workflow (run this cell for scheduled updates)

# 1. Reload latest data from Lakehouse
df_latest_update = spark.read.table("MaintenanceML.machine_data_raw").toPandas()

# 2. Apply same feature engineering (reuse code from Parts 2-5)
# ... (feature engineering code)

# 3. Load model and generate predictions
predictions_update = loaded_model.predict(X_predict)

# 4. Save updated predictions
# ... (save to Lakehouse)

print("✅ Daily prediction update completed")
print(f"Timestamp: {datetime.utcnow()}")

In [ ]:
# Load new CSV data (e.g., sample_quality_data.csv from IoT simulator)
csv_file = "Files/sample_quality_data.csv"  # Upload this file to Lakehouse Files first

# Read CSV directly to pandas
df_new = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(csv_file) \
    .toPandas()

# Convert timestamp to datetime
df_new['Timestamp'] = pd.to_datetime(df_new['Timestamp'])
df_new = df_new.sort_values(['MachineID', 'Timestamp']).reset_index(drop=True)

print(f"✅ Loaded {len(df_new):,} new records")
print(f"Date range: {df_new['Timestamp'].min()} to {df_new['Timestamp'].max()}")
print(f"Machines: {df_new['MachineID'].nunique()}")
df_new.head()

## Part 10.2: Feature Engineering on New Data

In [ ]:
# Apply same feature engineering as training data

# Step 1: Calculate rotation count per cycle
df_new['RotationCount'] = df_new['ActualAngle'] / 360.0

# Step 2: Cumulative rotations per machine
df_new['CumulativeRotation'] = df_new.groupby('MachineID')['RotationCount'].cumsum()

# Step 3: Use BitRotationCounter if available
if 'BitRotationCounter' in df_new.columns:
    df_new['CumulativeBitRotation'] = df_new['BitRotationCounter']
else:
    df_new['CumulativeBitRotation'] = df_new['CumulativeRotation']

# Step 4: Rolling window features (1 hour)
df_new_indexed = df_new.set_index('Timestamp')
rolling_features_new = df_new_indexed.groupby('MachineID').rolling('1H').agg({
    'RotationCount': ['sum', 'mean'],
    'ActualTorque': ['mean', 'std'],
    'CycleTime_ms': ['mean', 'max'],
    'CycleOK': 'mean'
}).reset_index()

rolling_features_new.columns = ['MachineID', 'Timestamp', 
                                 'Rot_LastHour_Sum', 'Rot_LastHour_Avg',
                                 'Torque_LastHour_Avg', 'Torque_LastHour_Std',
                                 'CycleTime_LastHour_Avg', 'CycleTime_LastHour_Max',
                                 'PassRate_LastHour']

df_new = df_new.merge(rolling_features_new, on=['MachineID', 'Timestamp'], how='left')
df_new = df_new.fillna(0)

# Step 5: Get latest state per machine
df_new_latest = df_new.sort_values('Timestamp').groupby('MachineID').tail(1).reset_index(drop=True)

print(f"✅ Feature engineering completed")
print(f"Ready to predict for {len(df_new_latest)} machines")
df_new_latest[['MachineID', 'CumulativeBitRotation', 'Rot_LastHour_Sum']].head()

## Part 10.3: Generate Predictions on New CSV Data

In [ ]:
# Prepare features for prediction (same features as training)
X_new_predict = df_new_latest[feature_cols]

# Make predictions using the loaded model
predictions_new = loaded_model.predict(X_new_predict)

# Add predictions to dataframe
df_new_latest['ML_PredictedDays'] = predictions_new

# Calculate risk level
df_new_latest['RiskLevel'] = pd.cut(
    df_new_latest['ML_PredictedDays'],
    bins=[0, 2, 7, 14, 365],
    labels=['🔴 CRITICAL', '🟠 URGENT', '🟡 WARNING', '🟢 GOOD']
)

# Add prediction timestamp
df_new_latest['PredictionTimestamp'] = datetime.utcnow()

# Calculate rule-based prediction for comparison
BIT_LIFETIME = 100000
df_new_latest['RemainingRotations'] = BIT_LIFETIME - df_new_latest['CumulativeBitRotation']
df_new_latest['RotationsPerHour'] = df_new_latest['Rot_LastHour_Sum']
df_new_latest['HoursUntilReplacement'] = df_new_latest['RemainingRotations'] / (df_new_latest['RotationsPerHour'] + 0.1)
df_new_latest['DaysUntilReplacement'] = (df_new_latest['HoursUntilReplacement'] / 24).clip(0, 365)

print("✅ Predictions generated on new CSV data")
print(f"\n📊 Risk Distribution:")
print(df_new_latest['RiskLevel'].value_counts())
print(f"\n🔍 Sample Predictions:")
df_new_latest[['MachineID', 'CumulativeBitRotation', 'DaysUntilReplacement', 'ML_PredictedDays', 'RiskLevel']].head(10)

## Part 10.4: Save New Predictions to Lakehouse

In [ ]:
# Select columns for new prediction table
prediction_cols_new = [
    'PredictionTimestamp', 'Timestamp', 'MachineID',
    'CumulativeBitRotation', 'RotationCount',
    'Rot_LastHour_Sum', 'PassRate_LastHour',
    'DaysUntilReplacement',  # Rule-based prediction
    'ML_PredictedDays',       # ML prediction
    'RiskLevel'
]

df_new_predictions = df_new_latest[prediction_cols_new].copy()

# Convert to Spark and save (append to existing predictions)
spark_new_predictions = spark.createDataFrame(df_new_predictions)
spark_new_predictions.write.mode("append").saveAsTable("MaintenanceML.ml_predictions")

print("✅ New predictions saved to: MaintenanceML.ml_predictions")
print(f"Added {len(df_new_predictions)} new predictions")
print(f"\n📈 Comparison: ML vs Rule-Based")
comparison = df_new_predictions[['MachineID', 'DaysUntilReplacement', 'ML_PredictedDays', 'RiskLevel']].head(10)
print(comparison.to_string(index=False))